In [ ]:
#installing tweepy library
!pip install tweepy

In [ ]:
#installing config
!pip install config

In [ ]:
#installing textblob
!pip install textblob

In [ ]:
#installing preprocessor
!pip install preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4477 sha256=67a40b9442ad252fe5086db3eaddefe504a15d494a65e95ae3855a8542592b7f
  Stored in directory: /root/.cache/pip/wheels/0e/b7/36/aa37256db62b4bfd35a6f1b5536e9ba843f257b79dcbf3d5f1
Successfully built preprocessor


Getting tweets

In [ ]:
#importing necessary libraries
from tweepy import OAuthHandler
from tweepy.streaming import Stream
import tweepy as tw
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

In [ ]:
#declaring the twitter consumer and access tokens
TWITTER_CONSUMER_KEY = 'CNOu5SorWOir7IkNn27WN8vx2'
TWITTER_CONSUMER_SECRET = 'ZlyEm7qNp7ELyhxkBiJyryO0zRRnOzEwrG7CrQkOCBhHv8CLm1'
TWITTER_ACCESS_TOKEN = '1403047363325796355-XVXZ5zUAEZtl2cFt3shXndGx57IE6S'
TWITTER_ACCESS_TOKEN_SECRET = 'DCnQpZIdL5xz4nie3FaF0ja3HRWg0irY84WL3iuTfsGwe'
auth = tw.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
#defining function to scrap tweets using api.search
def scraptweets(search_words, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tw.Cursor(api.search, q=search_words, lang="en", tweet_mode='extended').items(numTweets)
        
# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
# Begin scraping the tweets individually:
        noTweets = 0
# Pull the values
        for tweet in tweet_list:
          username = tweet.user.screen_name
          acctdesc = tweet.user.description
          location = tweet.user.location
          following = tweet.user.friends_count
          followers = tweet.user.followers_count
          totaltweets = tweet.user.statuses_count
          usercreatedts = tweet.user.created_at
          tweetcreatedts = tweet.created_at
          retweetcount = tweet.retweet_count
          hashtags = tweet.entities['hashtags']
          try:
            text = tweet.retweeted_status.full_text
# Not a Retweet
          except AttributeError:  
            text = tweet.full_text
# Add the 11 variables to the empty list - ith_tweet:
          ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
# Append to dataframe - db_tweets
          db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
          noTweets += 1
        
        # Run ended:
          end_run = time.time()
          duration_run = round((end_run-start_run)/60, 2)
        
          print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
          print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) #15 minute sleep time
# Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
# Define working path and filename
    #path = os.getcwd()
    filename = '/content/drive/MyDrive/Big_Data_Project/tweets.csv'
# Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [ ]:
#initialise function parameters
search_words = "coke -filter:retweets"
#search_words = "pepsi -filter:retweets"
#search_words = "cokevspepsi -filter:retweets"
#search_words = "pepsivscoke -filter:retweets"
#date_since = "2003-01-01"
numTweets = 2500
numRuns = 8
#call the function scraptweets
scraptweets(search_words, numTweets, numRuns)